# Summarization of XLSum news by Fine Tuning T5 Transformer

In [5]:
!pip install sentencepiece

In [6]:
!pip install transformers -q
!pip install wandb -q
!pip install rouge

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

dataset = load_dataset('csebuetnlp/xlsum', 'arabic')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/arabic/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import io
import json

from rouge import Rouge

from transformers import T5Tokenizer, T5ForConditionalGeneration
import wandb

import csv
import xml.etree.ElementTree as ET

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [7]:
# Specify the path to your XML file
xml_file_path = "/kaggle/input/challenge-dataset-training/BAH_waseet_news_1.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element of the XML
root = tree.getroot()
# Create a list to store the extracted data
data = []
# Iterate over each 'item' element in the XML
for item in root.findall('item'):
    address = item.find('address').text
    title = item.find('title').text
    article = item.find('article').text
    summary = item.find('summary').text
    
    # Append the data as a dictionary to the list
    data.append({
        'address': address,
        'title': title,
        'article': article,
        'summary': summary
    })
data = pd.DataFrame(data)

In [8]:
data

,address,title,article,summary
0,http://www.alwasatnews.com/news/764788.html,الاف بي آي لم يكن يعلم باي تهديدات قبل الماراثون,الاف بي آي لم يكن يعلم باي تهديدات قبل الماراثون,وقال المسؤول في الاف بي آي ريك ديلورييه في مؤت...
1,http://www.alwasatnews.com/news/764780.html,انفجارات بوسطن تثير صدمة في المجتمع الرياضي.. ...,انفجارات بوسطن تثير صدمة في المجتمع الرياضي.. ...,من يستطيع أن يفعل شيئا كهذا .وأوضحت العداءة ال...
2,http://www.alwasatnews.com/news/764787.html,كوريا الشمالية تهدد سيول مجددا برد وشيك,وياتي هذا التحذير غداة اختتام وزير الخارجية ال...,واعربت كوريا الجنوبية عن اسفها للانذار الذي وج...
3,http://www.alwasatnews.com/news/764779.html,صندوق النقد الدولي وتونس يقتربان من اتفاق قرض ...,وتابع سنواصل التفاوض في الايام القادم من اجل ا...,وقال امين ماتي المدير المساعد بقسم الشرق الاوس...
4,http://www.alwasatnews.com/news/765067.html,الضحية الثالثة لتفجيري بوسطن صينية الجنسية,الضحية الثالثة لتفجيري بوسطن صينية الجنسية,وقالت القنصلية العامة للصين في نيويورك مساء ال...
...,...,...,...,...
23856,http://www.alwasatnews.com/news/775608.html,رسالة إلى... مراهق,ومع زيادة تدفق الهرمونات والأفكار معاً، تبدو ل...,يُعتبَر عمر المراهقة الجسر الذي نمر به بين الط...
23857,http://www.alwasatnews.com/news/765054.html,الزراعة الجزئية للقرنية,فعملية الزراعة الجزئية للقرنية حسب ما بين أيدي...,تؤكد الإحصاءات العلمية أن عملية الزراعة الجزئي...
23858,http://www.alwasatnews.com/news/775607.html,ماما... من أين يأتي الأطفال؟,ويُعتبَر سؤال الأطفال الأشهر بين عمر إلى سنوات...,يكبر الأطفال ويتعلمون المشي والكلام، ومع الوقت...
23859,http://www.alwasatnews.com/news/775603.html,العلاج بالوهم,العلاج بالوهم,يستخدم الأطباء مصطلح بلاسيبو وهي كلمة لاتينية ...


In [6]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [9]:
# determine the device we will be using for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))
#print('There are %d GPU(s) available.' % torch.cuda.device_count())

[INFO] training using Tesla P100-PCIE-16GB


In [10]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.article = self.data.article
        self.summary = self.data.summary

    def __len__(self):
        return len(self.article)

    def __getitem__(self, index):
        summary = str(self.summary[index])
        summary = ' '.join(summary.split())

        text = str(self.article[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([summary], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        #target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([self.summary[index]], max_length=self.summ_len, pad_to_max_length=True, return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [11]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]

        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [12]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)


            # generated_ids = model.generate(
            #     input_ids = ids,
            #     attention_mask = mask,
            #     max_length=150,
            #     num_beams=2,
            #     repetition_penalty=2.5,
            #     length_penalty=1.0,
            #     early_stopping=True
            #     )

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                num_beams=2,
                #max_length=config.SUMMARY_LEN,
                max_length=150,
                no_repeat_ngram_size=2,
                early_stopping=True
            )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')



            predictions.extend(preds)
            actuals.extend(target)

        if not predictions:
            raise ValueError("Predictions are empty")
        if not actuals:
            raise ValueError("Actuals are empty")

    # result = []
    # for i in range(len(predictions)):
    #     result.append({'example_id': example_ids[i], 'generated_summary': predictions[i], 'actual_text': actuals[i]})

    # # Save the generated summaries to a JSONL file
    # with open('generated_summaries.jsonl', 'w') as f:
    #     for r in result:
    #         f.write(json.dumps(r) + '\n')

#     result = []
#     for i in range(len(predictions)):
#         result.append({'generated_summary': predictions[i]})

#     # Save the generated summaries to a JSONL file
#     with open('summaries.jsonl', 'w') as f:
#         for r in result:
#             f.write(json.dumps(r) + '\n')

    rouge = Rouge()
    scores = rouge.get_scores(predictions, actuals, avg=True)
    print({'Rouge Results ':scores})
    return predictions, actuals

In [13]:
wandb.init(project="transformers_tutorials_summarization")
config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 2      # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = 512
config.SUMMARY_LEN = 150

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config.SEED) # pytorch random seed
np.random.seed(config.SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
#tokenizer = T5Tokenizer.from_pretrained("t5-base")

tokenizer = T5Tokenizer.from_pretrained("malmarjeh/t5-arabic-text-summarization")

#df = pd.read_csv(io.BytesIO(uploaded['news_summary.csv']),encoding='latin-1')
#df = pd.DataFrame(dataset['train'])
#df = data
df = data[['article','summary']]
df.summary = 'summarize: ' + df.summary

# val_df = pd.DataFrame(dataset['test'])
# val_df = df[['text','summary']]
# val_df.summary = 'summarize: ' + val_df.summary
#summary = 'summarize: ' + df.summary
print(df.head())
train_size = 0.5
train_dataset=df.sample(frac=train_size,random_state = config.SEED)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

# train_dataset=df.iloc[:20000]
#val_dataset=val_df.iloc[:4000]
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

#model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("malmarjeh/t5-arabic-text-summarization")
model = model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

wandb.watch(model, log="all")

print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/tmp/ipykernel_28/2448762773.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.summary = 'summarize: ' + df.summary


                                             article  \
0  الاف بي آي لم يكن يعلم باي تهديدات قبل الماراثون    
1  انفجارات بوسطن تثير صدمة في المجتمع الرياضي.. ...   
2  وياتي هذا التحذير غداة اختتام وزير الخارجية ال...   
3  وتابع سنواصل التفاوض في الايام القادم من اجل ا...   
4        الضحية الثالثة لتفجيري بوسطن صينية الجنسية    

                                             summary  
0  summarize: وقال المسؤول في الاف بي آي ريك ديلو...  
1  summarize: من يستطيع أن يفعل شيئا كهذا .وأوضحت...  
2  summarize: واعربت كوريا الجنوبية عن اسفها للان...  
3  summarize: وقال امين ماتي المدير المساعد بقسم ...  
4  summarize: وقالت القنصلية العامة للصين في نيوي...  
FULL Dataset: (23861, 2)
TRAIN Dataset: (11930, 2)
TEST Dataset: (11931, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  9.579279899597168
Epoch: 0, Loss:  0.2427373230457306
Epoch: 0, Loss:  2.372243881225586
Epoch: 0, Loss:  0.06378002464771271
Epoch: 0, Loss:  0.24157069623470306
Epoch: 0, Loss:  0.11597660183906555
Epoch: 0, Loss:  0.053042974323034286
Epoch: 0, Loss:  0.8225440979003906
Epoch: 0, Loss:  0.2694956064224243
Epoch: 0, Loss:  0.08637794852256775
Epoch: 0, Loss:  0.528222382068634
Epoch: 0, Loss:  0.0014341982314363122
Epoch: 1, Loss:  0.021056029945611954
Epoch: 1, Loss:  0.05184910073876381
Epoch: 1, Loss:  0.2890913188457489
Epoch: 1, Loss:  0.10694228857755661
Epoch: 1, Loss:  0.007337978575378656
Epoch: 1, Loss:  0.004362678620964289
Epoch: 1, Loss:  0.04443394020199776
Epoch: 1, Loss:  0.04943191260099411
Epoch: 1, Loss:  0.2551579475402832
Epoch: 1, Loss:  0.021933725103735924
Epoch: 1, Loss:  0.03889632597565651
Epoch: 1, Loss:  0.06431134045124054


In [12]:
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)

    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    print('Output Files generated for review')

print(final_df)

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1600
Completed 1700
Completed 1800
Completed 1900
{'Rouge Results ': {'rouge-1': {'r': 0.9173588137464139, 'p': 0.985221288965852, 'f': 0.9486555521301454}, 'rouge-2': {'r': 0.8742131849676503, 'p': 0.9769569836996722, 'f': 0.9207677012313913}, 'rouge-l': {'r': 0.9168927079407653, 'p': 0.9847335740793033, 'f': 0.9481790583931469}}}
Output Files generated for review
                                         Generated Text  \
0        القوات الأوكرانية الانسحاب من شبه جزيرة القرم.   
1     "هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...   
2     الشرطة في القطاع الهندي من إقليم كشمير إن انفج...   
3     راهب يدعى كوكاي، في المنحدرات كثيفة الأشجار بج.

In [14]:
model.save_pretrained("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/spiece.model',
 '/kaggle/working/added_tokens.json')

In [14]:

# %env JOBLIB_TEMP_FOLDER=/tmp
# import gc
# gc.collect()


In [15]:
with open('/kaggle/input/mit-challenge-validation/validation_data.jsonl', 'r') as f:
    val_lines = f.readlines()

# Parse each line as JSON and extract text
val_paragraphs = []
val_example_ids = []
val_summarys = []

for line in val_lines:
    data = json.loads(line)
    paragraph = data['paragraph']
    val_paragraphs.append(paragraph)

    example_id = data['example_id']
    val_example_ids.append(example_id)

    #summary = data['summary']
    #val_summarys.append(summary)

val_df__ = pd.DataFrame(val_example_ids, columns=['id'])
val_df__["Text"] = val_paragraphs
#val_df__ ["Summary"] = val_summarys

val_df__

,id,Text
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...
...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...


In [16]:
#model_name = "malmarjeh/t5-arabic-text-summarization"
# model_name = "csebuetnlp/mT5_multilingual_XLSum"
# abstractive_Tokenizer = AutoTokenizer.from_pretrained(model_name)
# abstractive_Model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Load the saved model and tokenizer
abstractive_Model = T5ForConditionalGeneration.from_pretrained("/kaggle/working/")
abstractive_Tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/")

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

abstractive_Model = abstractive_Model.to(device)

def abstractiveModelSummarizer(text):
    input_ids = abstractive_Tokenizer(
        [(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=1024
    )["input_ids"].to(device)

    output_ids = abstractive_Model.generate(
        input_ids=input_ids,
        max_length=800,
        no_repeat_ngram_size=8,
        num_beams=4
    )[0]

    summary = abstractive_Tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary


def evaluate_model(model, tokenizer, test_dataset):
    summaries = []
    #references = []

    for idx, row in test_dataset.iterrows():
        text = row['Text']
        #summary = row['Summary']
        generated_summary = abstractiveModelSummarizer(text)

        summaries.append(generated_summary)
        ##references.append(summary)
        print('index: ', idx)

        print(f"Input Text: {text}")
        print(f"Generated Summary: {generated_summary}")
        #print(f"Reference Summary: {summary}")
        print("-" * 50)

    return summaries

summaries = evaluate_model(abstractive_Model, abstractive_Tokenizer, val_df__)

# Create a list of dictionaries with index and summary fields
summaries_dict = [{"index": i, "summary": summary} for i, summary in enumerate(summaries)]

index:  0
Input Text: وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام تعال معا نتناول بيت المقدس بوصفه نموذجا لفتوحات هذه البلاد بشيء من التفصيل .
فتح بيت المقدس ١٥هـ / ٦٣٦م:
لحق عمرو بن العاص بالروم، وشدد عليهم الحصار ، ولم يتمكن الروم من فكه، وهرب الأرطبون إلى مصر مركز ثقل الوجود البيزنطي في إفريقيا لصد هجمات المسلمين.
وأثناء الحصار كانت تجرى محادثات لدخول أهل بيت المقدس في الصلح مع المسلمين، فلم يجد بطريرك بيت المقدس أمامه سوى المصالحة، ولكنه اشترط أن يتم الصلح مع الخليفة عمر بن الخطاب بنفسه وذلك لما للمدينة من قدسية وأهمية دينية كبيرة، فأرسل عمرو بن العاص بهذه الأخبار إلى الخليفة بالمدينة النبوية، وقال له في رسالته: "إني أعالج حربا كؤدًا صدومًا وبلادا ادخرت لك فلك رأيك" فكان أول خروج للخليفة من المدينة بعد موت رسول الله ، هذا الخروج الأجل لكي يفتح بيت المقدس . وكان أول نزول له بمدن الشام بالجابية، والتي وفدت عليه فيها رسل البطريرك طالبين الصلح، فصالحهم عمر صلحا لم تحظ به مدينة أخرى في البلاد المفتوحة.... 
هل تعرف شروط هذا الصلح ؟
اقرأ النص التالى لتتعرف كيف كان الخليفة عمر سخيا في م

In [18]:
# Write each summary as a separate line in the output file
with open('/kaggle/working/summaries.jsonl', 'w', encoding='utf-8') as f:
    for i, summary in enumerate(summaries):
        summary_dict = {"example_id": i, "summary": summary}
        json.dump(summary_dict, f, ensure_ascii=False)
        f.write('\n')